In [5]:
import flwr as fl
from pathlib import Path
import os

from sources.simulation_framework.base_simulator import BaseSimulator
from sources.datasets.femnist.femnist_client_dataset_factory import FemnistClientDatasetFactory
from sources.flwr_parameters.default_parameters import DEFAULT_SEED
from sources.models.femnist.femnist_model_template import FemnistModelTemplate

%load_ext autoreload
%autoreload 2

ImportError: cannot import name 'BaseSimulator' from 'sources.simulation_framework.base_simulator' (/home/marukku/Documents/CST_PartII_Project/Project_Files/impact_of_non_iid_data_in_federated_learning/sources/simulation_framework/base_simulator.py)

In [6]:
base_dir = Path(os.getcwd()).parent.parent / "data"

In [7]:
def eval_config(rnd: int):
    """Return training configuration dict for each round.
    Keep batch size fixed at 32, perform two rounds of training with one
    local epoch, increase to two local epochs afterwards.
    """
    config = {
        "batch_size": 32,
        "val_steps": 2
    }
    return config

def fit_config(rnd: int):
    """Return training configuration dict for each round.
    Keep batch size fixed at 32, perform two rounds of training with one
    local epoch, increase to two local epochs afterwards.
    """
    config = {
        "batch_size": 32,
        "local_epochs": 1 if rnd < 2 else 2,
    }
    return config

simulation_parameters = {"num_rounds": 2, "num_clients": 2}
strategy = fl.server.strategy.FedAvg(on_fit_config_fn=fit_config, on_evaluate_config_fn=eval_config)
model_template = FemnistModelTemplate(DEFAULT_SEED)
dataset_factory = FemnistClientDatasetFactory(base_dir)

simulator = BaseSimulator(simulation_parameters, strategy, model_template, dataset_factory)

NameError: name 'BaseSimulator' is not defined

In [4]:
from sources.flwr_parameters.simulation_parameters import DEFAULT_SERVER_ADDRESS

fl.server.start_server(server_address=DEFAULT_SERVER_ADDRESS, strategy=strategy, config={"num_rounds": simulation_parameters["num_rounds"]})

INFO flower 2021-12-01 23:39:50,055 | app.py:77 | Flower server running (insecure, 2 rounds)
INFO flower 2021-12-01 23:39:50,056 | server.py:118 | Initializing global parameters
INFO flower 2021-12-01 23:39:50,056 | server.py:304 | Requesting initial parameters from one random client
INFO flower 2021-12-01 23:39:56,198 | server.py:307 | Received initial parameters from one random client
INFO flower 2021-12-01 23:39:56,200 | server.py:120 | Evaluating initial parameters
INFO flower 2021-12-01 23:39:56,201 | server.py:133 | FL starting
DEBUG flower 2021-12-01 23:39:58,451 | server.py:251 | fit_round: strategy sampled 2 clients (out of 2)
DEBUG flower 2021-12-01 23:39:59,534 | server.py:260 | fit_round received 2 results and 0 failures
DEBUG flower 2021-12-01 23:39:59,578 | server.py:201 | evaluate_round: strategy sampled 2 clients (out of 2)
DEBUG flower 2021-12-01 23:39:59,767 | server.py:210 | evaluate_round received 2 results and 0 failures
DEBUG flower 2021-12-01 23:39:59,768 | serve

In [ ]:
simulator.start_simulation()

INFO flower 2021-12-02 14:23:30,082 | app.py:77 | Flower server running (insecure, 2 rounds)
INFO flower 2021-12-02 14:23:30,084 | server.py:118 | Initializing global parameters
INFO flower 2021-12-02 14:23:30,086 | server.py:304 | Requesting initial parameters from one random client
2021-12-02 14:23:32.121967: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 14:23:32.122589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2021-12-02 14:23:32.122690: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2